# Analsysis and Data Clean and Data Export
Performing ETL on source data

### Import Dependencies

In [114]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd

from scipy import stats
from scipy.stats import linregress

from datetime import datetime


import requests
import json
from pprint import pprint


import sqlite3

from flask import Flask, jsonify, render_template, request
from sqlalchemy import create_engine, text, inspect
from sqlalchemy.orm import Session

import sqlalchemy
from sqlalchemy.ext.automap import automap_base


### Read in Police Data

In [115]:

months_dict = {
    "2020-10":"Oct 2020",
    "2020-11":"Nov 2020",
    "2020-12":"Dec 2020",
    "2021-01":"Jan 2021",
    "2021-02":"Feb 2021",
    "2021-03":"Mar 2021",
    "2021-04":"Apr 2021",
    "2021-05":"May 2021",
    "2021-06":"Jun 2021",
    "2021-07":"Jul 2021",
    "2021-08":"Aug 2021",
    "2021-09":"Sep 2021",
    "2021-10":"Oct 2021",
    "2021-11":"Nov 2021",
    "2021-12":"Dec 2021",
    "2022-01":"Jan 2022",
    "2022-02":"Feb 2022",
    "2022-03":"Mar 2022",
    "2022-04":"Apr 2022",
    "2022-05":"May 2022",
    "2022-06":"Jun 2022",
    "2022-07":"Jul 2022",
    "2022-08":"Aug 2022",
    "2022-09":"Sep 2022",
    "2022-10":"Oct 2022",
    "2022-11":"Nov 2022",
    "2022-12":"Dec 2022",
    "2023-01":"Jan 2023",
    "2023-02":"Feb 2023",
    "2023-03":"Mar 2023",
    "2023-04":"Apr 2023",
    "2023-05":"May 2023",
    "2023-06":"Jun 2023",
    "2023-07":"Jul 2023",
    "2023-08":"Aug 2023",
    "2023-09":"Sep 2023",    
}


for x in range(len(months_dict)):

    month = list(months_dict.keys())[x]

    df_current = pd.read_csv(f"Resources/PoliceData/{month}/{month}-west-midlands-street.csv")

    if x == 0:
        police_df = df_current

    if x != 0:
        police_df = pd.concat([police_df,df_current])


police_df.head()


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,bd82318efc6171ac49a4dbec0b8e5e4a11d79f89c0f3e6...,2020-10,West Midlands Police,West Midlands Police,-0.204743,51.625449,On or near Park/Open Space,E01000279,Barnet 007C,Violence and sexual offences,Investigation complete; no suspect identified,NaN
1,f780cb03e04bcbf9f30d2346e144cb3c1bfa379f200278...,2020-10,West Midlands Police,West Midlands Police,-0.195913,51.621132,On or near Shortgate,E01000276,Barnet 020A,Drugs,Status update unavailable,NaN
2,a645a44b40e005773f0aced2555044a2eb44590af981e2...,2020-10,West Midlands Police,West Midlands Police,-0.195913,51.621132,On or near Shortgate,E01000276,Barnet 020A,Other theft,Investigation complete; no suspect identified,NaN
3,4bf3715364357cbe4119e292e922df51b4f2906eb5e4ab...,2020-10,West Midlands Police,West Midlands Police,-0.199065,51.600273,On or near Priory Close,E01000204,Barnet 025C,Vehicle crime,Investigation complete; no suspect identified,NaN
4,NaN,2020-10,West Midlands Police,West Midlands Police,-1.841944,52.597265,On or near Chelsea Drive,E01009418,Birmingham 001B,Anti-social behaviour,NaN,NaN


In [116]:
len(police_df)

1136637

In [117]:
cleaned = police_df.loc[:,["LSOA code","LSOA name","Month","Longitude","Latitude","Crime type"]].copy()
cleaned.head()

,LSOA code,LSOA name,Month,Longitude,Latitude,Crime type
0,E01000279,Barnet 007C,2020-10,-0.204743,51.625449,Violence and sexual offences
1,E01000276,Barnet 020A,2020-10,-0.195913,51.621132,Drugs
2,E01000276,Barnet 020A,2020-10,-0.195913,51.621132,Other theft
3,E01000204,Barnet 025C,2020-10,-0.199065,51.600273,Vehicle crime
4,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour


In [118]:
crime_types = list(cleaned["Crime type"].unique())
crime_types

['Violence and sexual offences',
 'Drugs',
 'Other theft',
 'Vehicle crime',
 'Anti-social behaviour',
 'Burglary',
 'Criminal damage and arson',
 'Public order',
 'Shoplifting',
 'Theft from the person',
 'Possession of weapons',
 'Other crime',
 'Bicycle theft',
 'Robbery']

In [119]:
crime_type_IDs = np.arange(1, 15)
crime_type_IDs = ['type' + str(id) for id in crime_type_IDs]
crime_type_IDs

['type1',
 'type2',
 'type3',
 'type4',
 'type5',
 'type6',
 'type7',
 'type8',
 'type9',
 'type10',
 'type11',
 'type12',
 'type13',
 'type14']

In [120]:
crime_type_df = pd.DataFrame(list(zip(crime_type_IDs,crime_types)),
                           columns=['crime_type_ID','Crime type'])
crime_type_df

,crime_type_ID,Crime type
0,type1,Violence and sexual offences
1,type2,Drugs
2,type3,Other theft
3,type4,Vehicle crime
4,type5,Anti-social behaviour
5,type6,Burglary
6,type7,Criminal damage and arson
7,type8,Public order
8,type9,Shoplifting
9,type10,Theft from the person


In [121]:
# crime_type_df.to_csv("Resources/crime_types.csv", index=False)


### Read in IMD Data

In [122]:
IMD_data = pd.read_csv("Resources/IMD_Data/IMD.csv")
IMD_data.head()

,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Income Rank (where 1 is most deprived),Income Decile (where 1 is most deprived 10% of LSOAs),...,Indoors Sub-domain Rank (where 1 is most deprived),Indoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Outdoors Sub-domain Score,Outdoors Sub-domain Rank (where 1 is most deprived),Outdoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Total population: mid 2015 (excluding prisoners),Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners)
0,E01000001,City of London 001A,E09000001,City of London,6.208,29199,9,0.007,32831,10,...,16364,5,1.503,1615,1,1296,175,656,465,715
1,E01000002,City of London 001B,E09000001,City of London,5.143,30379,10,0.034,29901,10,...,22676,7,1.196,2969,1,1156,182,580,394,620
2,E01000003,City of London 001C,E09000001,City of London,19.402,14915,5,0.086,18510,6,...,17318,6,2.207,162,1,1350,146,759,445,804
3,E01000005,City of London 001E,E09000001,City of London,28.652,8678,3,0.211,6029,2,...,25218,8,1.769,849,1,1121,229,692,200,683
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,19.837,14486,5,0.117,14023,5,...,14745,5,0.969,4368,2,2040,522,1297,221,1285


In [123]:
IMD_data = IMD_data.iloc[:, [0,1,2,3,4,5,6,7,52]]
IMD_data.head()

,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners)
0,E01000001,City of London 001A,E09000001,City of London,6.208,29199,9,0.007,1296
1,E01000002,City of London 001B,E09000001,City of London,5.143,30379,10,0.034,1156
2,E01000003,City of London 001C,E09000001,City of London,19.402,14915,5,0.086,1350
3,E01000005,City of London 001E,E09000001,City of London,28.652,8678,3,0.211,1121
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,19.837,14486,5,0.117,2040


In [124]:
print(IMD_data['LSOA code (2011)'].nunique())
print(IMD_data['LSOA name (2011)'].nunique())

32844
32844


In [125]:
len(IMD_data)

32844

In [126]:
cleaned_IMD = IMD_data.rename(columns={'LSOA code (2011)':'LSOA code','LSOA name (2011)':'LSOA name'})
cleaned_IMD.head()

,LSOA code,LSOA name,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners)
0,E01000001,City of London 001A,E09000001,City of London,6.208,29199,9,0.007,1296
1,E01000002,City of London 001B,E09000001,City of London,5.143,30379,10,0.034,1156
2,E01000003,City of London 001C,E09000001,City of London,19.402,14915,5,0.086,1350
3,E01000005,City of London 001E,E09000001,City of London,28.652,8678,3,0.211,1121
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,19.837,14486,5,0.117,2040


### Merge IMD Data with Police Data

Perform Merge so that no loss occurs in Police Data - check against length of police data

In [127]:
Police_IMD_Merge = pd.merge(cleaned,cleaned_IMD,how='left',on="LSOA code")
print(len(Police_IMD_Merge))
Police_IMD_Merge.head()

1136637


,LSOA code,LSOA name_x,Month,Longitude,Latitude,Crime type,LSOA name_y,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners)
0,E01000279,Barnet 007C,2020-10,-0.204743,51.625449,Violence and sexual offences,Barnet 007C,E09000003,Barnet,8.956,25843.0,8.0,0.017,1511.0
1,E01000276,Barnet 020A,2020-10,-0.195913,51.621132,Drugs,Barnet 020A,E09000003,Barnet,4.061,31443.0,10.0,0.027,1494.0
2,E01000276,Barnet 020A,2020-10,-0.195913,51.621132,Other theft,Barnet 020A,E09000003,Barnet,4.061,31443.0,10.0,0.027,1494.0
3,E01000204,Barnet 025C,2020-10,-0.199065,51.600273,Vehicle crime,Barnet 025C,E09000003,Barnet,10.063,24436.0,8.0,0.068,1737.0
4,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,Birmingham 001B,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0


In [128]:
# Find unique LA districts using merged DF
Police_IMD_Merge['Local Authority District name (2019)'].unique()

array(['Barnet', 'Birmingham', 'Bournemouth, Christchurch and Poole',
       'Breckland', 'Bristol, City of', 'Bromsgrove',
       'Cheshire West and Chester', nan, 'Cotswold', 'County Durham',
       'Coventry', 'Daventry', 'Dudley', 'East Hampshire', 'Fylde',
       'Harborough', 'Hartlepool', 'Hinckley and Bosworth',
       'Isle of Wight', 'Kettering', 'Lichfield', 'North Warwickshire',
       'Nuneaton and Bedworth', 'Pendle', 'Richmondshire', 'Rugby',
       'Sandwell', 'Shropshire', 'Solihull', 'South Staffordshire',
       'Tewkesbury', 'Walsall', 'Warwick', 'Westminster',
       'West Oxfordshire', 'Wiltshire', 'Wolverhampton', 'Wychavon',
       'Ashford', 'Cheshire East', 'Craven', 'Derbyshire Dales', 'Dorset',
       'East Riding of Yorkshire', 'Kensington and Chelsea',
       'Malvern Hills', 'Northumberland', 'Oxford', 'Portsmouth',
       'Stratford-on-Avon', 'Swindon', 'Test Valley', 'Welwyn Hatfield',
       'West Berkshire', 'East Devon', 'Halton', 'Hammersmith and Fu

### Read file from Geolookups

Read files from Geolookups using GeoPandas - we will use this file to filter out any LSOAs which do not fall within the West Midlands Combined Authority boundaries that is found in the Police Data

In [129]:
gpd = gpd.read_file('Resources/geo_lookups/E47000007.geojson')

In [130]:
gpd.head()

,FID,LSOA11CD,LSOA11NM,LSOA11NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,LADCD_ACTIVE,UTLACD,CAUTHCD,RGNCD,geometry
0,8649,E01008881,Birmingham 067A,Birmingham 067A,412690,285442,-1.81463,52.46678,f5489416-627d-4ee3-b843-56e49150147a,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.81043 52.47024, -1.80946 52.47007..."
1,8650,E01008882,Birmingham 066A,Birmingham 066A,411842,285398,-1.82711,52.46641,febd9b79-50bf-467a-be7f-737ea8729ecf,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82823 52.46907, -1.82639 52.46867..."
2,8651,E01008883,Birmingham 078A,Birmingham 078A,412126,285001,-1.82294,52.46283,20bae2a8-8ba2-41a7-9da8-577593700ca3,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82277 52.46638, -1.82143 52.46632..."
3,8652,E01008884,Birmingham 078B,Birmingham 078B,411241,284714,-1.83598,52.46027,5a661dd5-081f-4c6d-aa1d-ba83f10dd4eb,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82565 52.46509, -1.82631 52.46453..."
4,8653,E01008885,Birmingham 076A,Birmingham 076A,413207,284799,-1.80704,52.46099,042c0c37-d270-4e14-969f-d17b8ff7628f,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.80395 52.46102, -1.80415 52.46083..."


In [131]:
# rename the columns to match before performing a merge

rename = gpd.rename(columns={'LSOA11CD':'LSOA code'})

rename.head()

,FID,LSOA code,LSOA11NM,LSOA11NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,LADCD_ACTIVE,UTLACD,CAUTHCD,RGNCD,geometry
0,8649,E01008881,Birmingham 067A,Birmingham 067A,412690,285442,-1.81463,52.46678,f5489416-627d-4ee3-b843-56e49150147a,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.81043 52.47024, -1.80946 52.47007..."
1,8650,E01008882,Birmingham 066A,Birmingham 066A,411842,285398,-1.82711,52.46641,febd9b79-50bf-467a-be7f-737ea8729ecf,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82823 52.46907, -1.82639 52.46867..."
2,8651,E01008883,Birmingham 078A,Birmingham 078A,412126,285001,-1.82294,52.46283,20bae2a8-8ba2-41a7-9da8-577593700ca3,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82277 52.46638, -1.82143 52.46632..."
3,8652,E01008884,Birmingham 078B,Birmingham 078B,411241,284714,-1.83598,52.46027,5a661dd5-081f-4c6d-aa1d-ba83f10dd4eb,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82565 52.46509, -1.82631 52.46453..."
4,8653,E01008885,Birmingham 076A,Birmingham 076A,413207,284799,-1.80704,52.46099,042c0c37-d270-4e14-969f-d17b8ff7628f,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.80395 52.46102, -1.80415 52.46083..."


In [132]:
print(len(rename))
print(rename['FID'].nunique())

1680
1680


In [133]:
# rename.to_csv("Resources/output/geo_lookup.csv", index=False)

In [134]:
# This merge will remove any LSOAs outside of WMCA in the police/IMD merge
merge_final = pd.merge(Police_IMD_Merge, rename, on='LSOA code')
print(len(merge_final))
merge_final.head()


1124466


,LSOA code,LSOA name_x,Month,Longitude,Latitude,Crime type,LSOA name_y,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,...,BNG_E,BNG_N,LONG,LAT,GlobalID,LADCD_ACTIVE,UTLACD,CAUTHCD,RGNCD,geometry
0,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.8376,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
1,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Burglary,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.8376,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
2,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Burglary,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.8376,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
3,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Criminal damage and arson,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.8376,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
4,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Public order,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.8376,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."


In [135]:
# check that only West Midlands LA Districts appear
merge_final['Local Authority District name (2019)'].unique()

array(['Birmingham', 'Coventry', 'Dudley', 'Sandwell', 'Solihull',
       'Walsall', 'Wolverhampton'], dtype=object)

In [136]:
# merge_final.to_csv("Resources/output/merge_test.csv", index=False)


### Clean the IMD Data using the geolookups boundary data

In [137]:

wmca_IMD = pd.merge(cleaned_IMD,rename, on='LSOA code')
wmca_IMD.head()

,LSOA code,LSOA name,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners),FID,...,BNG_E,BNG_N,LONG,LAT,GlobalID,LADCD_ACTIVE,UTLACD,CAUTHCD,RGNCD,geometry
0,E01008881,Birmingham 067A,E08000025,Birmingham,41.179,3924,2,0.232,1642,8649,...,412690,285442,-1.81463,52.46678,f5489416-627d-4ee3-b843-56e49150147a,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.81043 52.47024, -1.80946 52.47007..."
1,E01008882,Birmingham 066A,E08000025,Birmingham,59.693,939,1,0.311,1769,8650,...,411842,285398,-1.82711,52.46641,febd9b79-50bf-467a-be7f-737ea8729ecf,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82823 52.46907, -1.82639 52.46867..."
2,E01008883,Birmingham 078A,E08000025,Birmingham,38.636,4684,2,0.230,2050,8651,...,412126,285001,-1.82294,52.46283,20bae2a8-8ba2-41a7-9da8-577593700ca3,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82277 52.46638, -1.82143 52.46632..."
3,E01008884,Birmingham 078B,E08000025,Birmingham,44.315,3204,1,0.273,2091,8652,...,411241,284714,-1.83598,52.46027,5a661dd5-081f-4c6d-aa1d-ba83f10dd4eb,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82565 52.46509, -1.82631 52.46453..."
4,E01008885,Birmingham 076A,E08000025,Birmingham,22.921,12110,4,0.102,1373,8653,...,413207,284799,-1.80704,52.46099,042c0c37-d270-4e14-969f-d17b8ff7628f,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.80395 52.46102, -1.80415 52.46083..."


In [138]:
# Length of this data should match the length of the geolookups data
len(wmca_IMD)

1680

In [139]:
wmca_IMD_clean = wmca_IMD.iloc[:,[0,1,2,3,4,5,6,7,8]]
wmca_IMD_clean

,LSOA code,LSOA name,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners)
0,E01008881,Birmingham 067A,E08000025,Birmingham,41.179,3924,2,0.232,1642
1,E01008882,Birmingham 066A,E08000025,Birmingham,59.693,939,1,0.311,1769
2,E01008883,Birmingham 078A,E08000025,Birmingham,38.636,4684,2,0.230,2050
3,E01008884,Birmingham 078B,E08000025,Birmingham,44.315,3204,1,0.273,2091
4,E01008885,Birmingham 076A,E08000025,Birmingham,22.921,12110,4,0.102,1373
...,...,...,...,...,...,...,...,...,...
1675,E01033646,Birmingham 031I,E08000025,Birmingham,64.138,583,1,0.292,1891
1676,E01033647,Birmingham 058E,E08000025,Birmingham,53.340,1674,1,0.306,1581
1677,E01033648,Birmingham 084F,E08000025,Birmingham,55.778,1375,1,0.351,2708
1678,E01033649,Birmingham 058F,E08000025,Birmingham,52.028,1870,1,0.322,1881


In [140]:
wmca_IMD_clean['LSOA name'].nunique()

1680

In [141]:
wmca_IMD_clean.columns

Index(['LSOA code', 'LSOA name', 'Local Authority District code (2019)',
       'Local Authority District name (2019)',
       'Index of Multiple Deprivation (IMD) Score',
       'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)',
       'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)',
       'Income Score (rate)',
       'Total population: mid 2015 (excluding prisoners)'],
      dtype='object')

In [142]:
# wmca_IMD_clean.to_csv("Resources/wmca_IMD_clean.csv", index=False)


In [143]:
merge_final

,LSOA code,LSOA name_x,Month,Longitude,Latitude,Crime type,LSOA name_y,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,...,BNG_E,BNG_N,LONG,LAT,GlobalID,LADCD_ACTIVE,UTLACD,CAUTHCD,RGNCD,geometry
0,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.83760,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
1,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Burglary,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.83760,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
2,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Burglary,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.83760,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
3,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Criminal damage and arson,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.83760,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
4,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Public order,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.83760,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124461,E01009670,Coventry 041D,2023-09,-1.519938,52.379433,Violence and sexual offences,Coventry 041D,E08000026,Coventry,5.325,...,432766,275706,-1.52007,52.37843,68b4a782-8a22-4307-8f02-5029f213c413,E08000026,E08000026,E47000007,E12000005,"POLYGON ((-1.51622 52.37949, -1.51598 52.37883..."
1124462,E01009670,Coventry 041D,2023-09,-1.520035,52.378606,Violence and sexual offences,Coventry 041D,E08000026,Coventry,5.325,...,432766,275706,-1.52007,52.37843,68b4a782-8a22-4307-8f02-5029f213c413,E08000026,E08000026,E47000007,E12000005,"POLYGON ((-1.51622 52.37949, -1.51598 52.37883..."
1124463,E01009670,Coventry 041D,2023-09,-1.523360,52.378125,Violence and sexual offences,Coventry 041D,E08000026,Coventry,5.325,...,432766,275706,-1.52007,52.37843,68b4a782-8a22-4307-8f02-5029f213c413,E08000026,E08000026,E47000007,E12000005,"POLYGON ((-1.51622 52.37949, -1.51598 52.37883..."
1124464,E01009670,Coventry 041D,2023-09,-1.523360,52.378125,Violence and sexual offences,Coventry 041D,E08000026,Coventry,5.325,...,432766,275706,-1.52007,52.37843,68b4a782-8a22-4307-8f02-5029f213c413,E08000026,E08000026,E47000007,E12000005,"POLYGON ((-1.51622 52.37949, -1.51598 52.37883..."


In [144]:
merge_final.head(2)

,LSOA code,LSOA name_x,Month,Longitude,Latitude,Crime type,LSOA name_y,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,...,BNG_E,BNG_N,LONG,LAT,GlobalID,LADCD_ACTIVE,UTLACD,CAUTHCD,RGNCD,geometry
0,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.8376,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."
1,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Burglary,Birmingham 001B,E08000025,Birmingham,19.086,...,411095,300516,-1.8376,52.60233,d962e710-4e97-43a1-a4a3-da4750ee2f18,E08000025,E08000025,E47000007,E12000005,"POLYGON ((-1.82782 52.60865, -1.82902 52.59980..."


In [145]:
merge_final.columns

Index(['LSOA code', 'LSOA name_x', 'Month', 'Longitude', 'Latitude',
       'Crime type', 'LSOA name_y', 'Local Authority District code (2019)',
       'Local Authority District name (2019)',
       'Index of Multiple Deprivation (IMD) Score',
       'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)',
       'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)',
       'Income Score (rate)',
       'Total population: mid 2015 (excluding prisoners)', 'FID', 'LSOA11NM',
       'LSOA11NMW', 'BNG_E', 'BNG_N', 'LONG', 'LAT', 'GlobalID',
       'LADCD_ACTIVE', 'UTLACD', 'CAUTHCD', 'RGNCD', 'geometry'],
      dtype='object')

In [146]:
# extract columsn which may be relevant
police_clean = merge_final.iloc[:,[0,1,2,3,4,5,7,8,9,10,11,12,13]]
police_clean.head(2)

,LSOA code,LSOA name_x,Month,Longitude,Latitude,Crime type,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners)
0,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0
1,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Burglary,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0


In [147]:
# police_clean2 = pd.merge(police_clean, crime_type_df, on='Crime type')
# police_clean2

In [148]:
# police_clean = police_clean2.drop(columns=['Crime type'])
# police_clean

In [149]:
# police_clean.columns

### Add our own crime ID to the data

In [150]:
crime_IDs = np.arange(1, len(police_clean)+1)
crime_IDs = ['id_' + str(id) for id in crime_IDs]

In [151]:
police_clean['crime_ID'] = crime_IDs
police_clean.head(2)

C:\Users\smmr8\AppData\Local\Temp\ipykernel_22628\3931737405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_clean['crime_ID'] = crime_IDs


,LSOA code,LSOA name_x,Month,Longitude,Latitude,Crime type,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners),crime_ID
0,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0,id_1
1,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Burglary,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0,id_2


In [152]:
# police_clean.to_csv("Resources/output/police_clean.csv", index=False)


In [153]:
# gdf = gpd.read_file('Resources/geo_lookups/E47000007.geojson')
# gdf.head()

In [154]:
# gpd.to_csv("Resources/output/geo_lookup_wm.csv", index=False)


In [155]:
print(wmca_IMD_clean['Local Authority District name (2019)'].unique())
wmca_IMD_clean.head()

['Birmingham' 'Coventry' 'Dudley' 'Sandwell' 'Solihull' 'Walsall'
 'Wolverhampton']


,LSOA code,LSOA name,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners)
0,E01008881,Birmingham 067A,E08000025,Birmingham,41.179,3924,2,0.232,1642
1,E01008882,Birmingham 066A,E08000025,Birmingham,59.693,939,1,0.311,1769
2,E01008883,Birmingham 078A,E08000025,Birmingham,38.636,4684,2,0.230,2050
3,E01008884,Birmingham 078B,E08000025,Birmingham,44.315,3204,1,0.273,2091
4,E01008885,Birmingham 076A,E08000025,Birmingham,22.921,12110,4,0.102,1373


In [156]:
# output_geojson_path = 'output.geojson'

In [157]:
# gdf.to_file(output_geojson_path, driver='GeoJSON')

In [158]:

# #https://stackoverflow.com/questions/76155297/attributeerror-module-pandas-has-no-attribute-int64index
# gdf["row_id"] = gdf.index + 1
# gdf.reset_index(drop=True, inplace=True)
# gdf.set_index("row_id", inplace = True)
# gdf.to_file(output_geojson_path, driver='GeoJSON')


In [159]:
# gdf = gdf.iloc[:,[1,6,7,13]]
# gdf.head()

In [160]:

# gdf.to_file(output_geojson_path, driver='GeoJSON')


In [161]:
# type(gdf)

In [162]:
# from osgeo import ogr

In [163]:
# gdf['geometry'] = gdf['geometry'].apply(lambda geom: ogr.CreateGeometryFromWkt(geom.wkt).ExportToWkt())


In [164]:
# engine = create_engine('sqlite:///gdf.sqlite')
# gdf.to_sql('gdf', engine, index=False, if_exists='replace')

### Rename columns for merged DF in prep for SQLite

In [165]:
police_clean.head(2)

,LSOA code,LSOA name_x,Month,Longitude,Latitude,Crime type,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners),crime_ID
0,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0,id_1
1,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Burglary,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0,id_2


In [166]:
police_clean = police_clean.rename(columns={'LSOA name_x':'LSOA name',
                                            'Local Authority District code (2019)':'LA District code',
                                            'Local Authority District name (2019)': 'LA District name',
                                            'Index of Multiple Deprivation (IMD) Score': 'IMD Score',
                                            'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)': 'IMD Rank',
                                            'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)': 'IMD Decile',
                                            'Income Score (rate)': 'Income Score',
                                            'Total population: mid 2015 (excluding prisoners)': 'Total population',
                                            'crime_ID': 'Crime ID'
                                            })
police_clean.head(2)

,LSOA code,LSOA name,Month,Longitude,Latitude,Crime type,LA District code,LA District name,IMD Score,IMD Rank,IMD Decile,Income Score,Total population,Crime ID
0,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0,id_1
1,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Burglary,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0,id_2


In [167]:
police_clean.columns

Index(['LSOA code', 'LSOA name', 'Month', 'Longitude', 'Latitude',
       'Crime type', 'LA District code', 'LA District name', 'IMD Score',
       'IMD Rank', 'IMD Decile', 'Income Score', 'Total population',
       'Crime ID'],
      dtype='object')

In [168]:
police_clean = police_clean.rename(columns={'LSOA code':'LSOA_code',
                                            'LSOA name':'LSOA_name',
                                            'Crime type':'crime_type',
                                            'LA District code':'LA_District_code',
                                            'LA District name': 'LA_District_name',
                                            'IMD Score': 'IMD_Score',
                                            'IMD Rank': 'IMD_Rank',
                                            'IMD Decile': 'IMD_Decile',
                                            'Income Score': 'Income_Score',
                                            'Total population': 'Total_population',
                                            'Crime ID': 'crime_ID'
                                            })

police_clean.head(2)

,LSOA_code,LSOA_name,Month,Longitude,Latitude,crime_type,LA_District_code,LA_District_name,IMD_Score,IMD_Rank,IMD_Decile,Income_Score,Total_population,crime_ID
0,E01009418,Birmingham 001B,2020-10,-1.841944,52.597265,Anti-social behaviour,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0,id_1
1,E01009418,Birmingham 001B,2020-10,-1.839063,52.597809,Burglary,E08000025,Birmingham,19.086,15161.0,5.0,0.124,1519.0,id_2


In [169]:
final_merged = police_clean[['crime_ID', 'LSOA_code', 'LSOA_name', 'LA_District_code', 'LA_District_name', \
                             'Month', 'crime_type', 'Longitude', 'Latitude', 'IMD_Score', 'IMD_Decile', 'Income_Score', 'Total_population']]
final_merged.head(2)

,crime_ID,LSOA_code,LSOA_name,LA_District_code,LA_District_name,Month,crime_type,Longitude,Latitude,IMD_Score,IMD_Decile,Income_Score,Total_population
0,id_1,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Anti-social behaviour,-1.841944,52.597265,19.086,5.0,0.124,1519.0
1,id_2,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Burglary,-1.839063,52.597809,19.086,5.0,0.124,1519.0


In [170]:
len(final_merged)

1124466

### Get Lat Long from Geolookups
these are central Lat/Long points to be used for plotting when required - not the lat/long that is from police data which is for specific crime locations

In [171]:
#Add central LAT LON points for LSOA
LSOA_UK_lat_lon = pd.read_csv("Resources/Census_Data/lsoa_latlong.csv")

LSOA_UK_lat_lon = LSOA_UK_lat_lon.rename(columns={"lsoa11cd":"LSOA_code",'latitude':'lat_c','longitude':'lon_c'})


final_merged2 = pd.merge(final_merged,LSOA_UK_lat_lon,on="LSOA_code")
print(len(final_merged2))

final_merged2.head()

1124466


,crime_ID,LSOA_code,LSOA_name,LA_District_code,LA_District_name,Month,crime_type,Longitude,Latitude,IMD_Score,IMD_Decile,Income_Score,Total_population,LSOACD,lat_c,lon_c
0,id_1,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Anti-social behaviour,-1.841944,52.597265,19.086,5.0,0.124,1519.0,E01009418,52.600857,-1.836545
1,id_2,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Burglary,-1.839063,52.597809,19.086,5.0,0.124,1519.0,E01009418,52.600857,-1.836545
2,id_3,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Burglary,-1.841944,52.597265,19.086,5.0,0.124,1519.0,E01009418,52.600857,-1.836545
3,id_4,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Criminal damage and arson,-1.841944,52.597265,19.086,5.0,0.124,1519.0,E01009418,52.600857,-1.836545
4,id_5,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Public order,-1.839063,52.597809,19.086,5.0,0.124,1519.0,E01009418,52.600857,-1.836545


In [172]:
final_merged2.drop(columns=['LSOACD'], inplace=True)
final_merged2.head()

,crime_ID,LSOA_code,LSOA_name,LA_District_code,LA_District_name,Month,crime_type,Longitude,Latitude,IMD_Score,IMD_Decile,Income_Score,Total_population,lat_c,lon_c
0,id_1,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Anti-social behaviour,-1.841944,52.597265,19.086,5.0,0.124,1519.0,52.600857,-1.836545
1,id_2,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Burglary,-1.839063,52.597809,19.086,5.0,0.124,1519.0,52.600857,-1.836545
2,id_3,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Burglary,-1.841944,52.597265,19.086,5.0,0.124,1519.0,52.600857,-1.836545
3,id_4,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Criminal damage and arson,-1.841944,52.597265,19.086,5.0,0.124,1519.0,52.600857,-1.836545
4,id_5,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Public order,-1.839063,52.597809,19.086,5.0,0.124,1519.0,52.600857,-1.836545


### Export file with District names from the IMD data in order to use for colouring LSOAs in diff districts accordingly

In [173]:
wmca_IMD_clean.head()

,LSOA code,LSOA name,Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Total population: mid 2015 (excluding prisoners)
0,E01008881,Birmingham 067A,E08000025,Birmingham,41.179,3924,2,0.232,1642
1,E01008882,Birmingham 066A,E08000025,Birmingham,59.693,939,1,0.311,1769
2,E01008883,Birmingham 078A,E08000025,Birmingham,38.636,4684,2,0.230,2050
3,E01008884,Birmingham 078B,E08000025,Birmingham,44.315,3204,1,0.273,2091
4,E01008885,Birmingham 076A,E08000025,Birmingham,22.921,12110,4,0.102,1373


In [174]:
# Produce a database of LSOAs in each District
districts = list(wmca_IMD_clean['Local Authority District name (2019)'].unique())
print(districts)

['Birmingham', 'Coventry', 'Dudley', 'Sandwell', 'Solihull', 'Walsall', 'Wolverhampton']


In [175]:
district_LSOA = {}

In [176]:
district_LSOA

{}

In [177]:
test = wmca_IMD_clean.loc[wmca_IMD_clean['Local Authority District name (2019)'] == 'Birmingham', ['LSOA code']]
test.head()
type(list(test))
print(test)
len(test)



      LSOA code
0     E01008881
1     E01008882
2     E01008883
3     E01008884
4     E01008885
...         ...
1675  E01033646
1676  E01033647
1677  E01033648
1678  E01033649
1679  E01033650

[639 rows x 1 columns]


639

In [178]:
test2 = wmca_IMD_clean.loc[wmca_IMD_clean['Local Authority District name (2019)'] == 'Sandwell', ['LSOA code']]
test2.head()
type(list(test2))
print(test2)
len(test2)


test_x  = test2.values.tolist()
len(test_x)

      LSOA code
975   E01009921
976   E01009922
977   E01009923
978   E01009924
979   E01009925
...         ...
1156  E01010105
1157  E01010106
1158  E01010107
1629  E01032592
1632  E01032887

[186 rows x 1 columns]


186

In [179]:
cov_val = wmca_IMD_clean.loc[wmca_IMD_clean['Local Authority District name (2019)'] == 'Coventry', ['LSOA code']].values.tolist()
# cov_val.tolist()
# print(list)

new_val = []
for val in cov_val:
    new_val.append(val[0])

new_val

['E01009522',
 'E01009523',
 'E01009524',
 'E01009525',
 'E01009526',
 'E01009527',
 'E01009528',
 'E01009529',
 'E01009530',
 'E01009531',
 'E01009532',
 'E01009535',
 'E01009536',
 'E01009537',
 'E01009538',
 'E01009539',
 'E01009540',
 'E01009541',
 'E01009542',
 'E01009543',
 'E01009544',
 'E01009548',
 'E01009549',
 'E01009550',
 'E01009552',
 'E01009553',
 'E01009554',
 'E01009555',
 'E01009556',
 'E01009557',
 'E01009558',
 'E01009559',
 'E01009560',
 'E01009561',
 'E01009562',
 'E01009563',
 'E01009564',
 'E01009565',
 'E01009566',
 'E01009567',
 'E01009568',
 'E01009569',
 'E01009570',
 'E01009571',
 'E01009572',
 'E01009573',
 'E01009574',
 'E01009575',
 'E01009576',
 'E01009577',
 'E01009578',
 'E01009579',
 'E01009580',
 'E01009581',
 'E01009582',
 'E01009583',
 'E01009584',
 'E01009585',
 'E01009586',
 'E01009587',
 'E01009588',
 'E01009589',
 'E01009590',
 'E01009591',
 'E01009592',
 'E01009593',
 'E01009594',
 'E01009596',
 'E01009597',
 'E01009598',
 'E01009599',
 'E010

In [180]:
district_LSOA = {k: [] for k in districts}
district_LSOA.keys() 
list1 = []
list2 = []
for district in districts:
     list1 = wmca_IMD_clean.loc[wmca_IMD_clean['Local Authority District name (2019)'] == district, ['LSOA code']].values.tolist()

     for element in list1:
          # list2.append(element[0])
          district_LSOA[district].append(element[0])


In [181]:
district_LSOA

{'Birmingham': ['E01008881',
  'E01008882',
  'E01008883',
  'E01008884',
  'E01008885',
  'E01008886',
  'E01008887',
  'E01008888',
  'E01008889',
  'E01008890',
  'E01008891',
  'E01008892',
  'E01008893',
  'E01008894',
  'E01008895',
  'E01008896',
  'E01008897',
  'E01008898',
  'E01008899',
  'E01008901',
  'E01008905',
  'E01008906',
  'E01008907',
  'E01008909',
  'E01008910',
  'E01008911',
  'E01008913',
  'E01008914',
  'E01008915',
  'E01008916',
  'E01008917',
  'E01008918',
  'E01008919',
  'E01008920',
  'E01008921',
  'E01008923',
  'E01008924',
  'E01008925',
  'E01008927',
  'E01008928',
  'E01008929',
  'E01008930',
  'E01008931',
  'E01008932',
  'E01008933',
  'E01008934',
  'E01008935',
  'E01008936',
  'E01008937',
  'E01008938',
  'E01008939',
  'E01008940',
  'E01008941',
  'E01008942',
  'E01008943',
  'E01008944',
  'E01008945',
  'E01008946',
  'E01008947',
  'E01008948',
  'E01008949',
  'E01008950',
  'E01008951',
  'E01008952',
  'E01008953',
  'E0100895

In [182]:
district_LSOA.keys()

dict_keys(['Birmingham', 'Coventry', 'Dudley', 'Sandwell', 'Solihull', 'Walsall', 'Wolverhampton'])

In [183]:
type(district_LSOA['Birmingham'][0])

str

In [184]:
district_LSOA['Birmingham']

['E01008881',
 'E01008882',
 'E01008883',
 'E01008884',
 'E01008885',
 'E01008886',
 'E01008887',
 'E01008888',
 'E01008889',
 'E01008890',
 'E01008891',
 'E01008892',
 'E01008893',
 'E01008894',
 'E01008895',
 'E01008896',
 'E01008897',
 'E01008898',
 'E01008899',
 'E01008901',
 'E01008905',
 'E01008906',
 'E01008907',
 'E01008909',
 'E01008910',
 'E01008911',
 'E01008913',
 'E01008914',
 'E01008915',
 'E01008916',
 'E01008917',
 'E01008918',
 'E01008919',
 'E01008920',
 'E01008921',
 'E01008923',
 'E01008924',
 'E01008925',
 'E01008927',
 'E01008928',
 'E01008929',
 'E01008930',
 'E01008931',
 'E01008932',
 'E01008933',
 'E01008934',
 'E01008935',
 'E01008936',
 'E01008937',
 'E01008938',
 'E01008939',
 'E01008940',
 'E01008941',
 'E01008942',
 'E01008943',
 'E01008944',
 'E01008945',
 'E01008946',
 'E01008947',
 'E01008948',
 'E01008949',
 'E01008950',
 'E01008951',
 'E01008952',
 'E01008953',
 'E01008954',
 'E01008955',
 'E01008956',
 'E01008957',
 'E01008958',
 'E01008959',
 'E010

In [185]:
type(district_LSOA)

dict

In [186]:
# check
total = 0
for district in districts:
    total += len(district_LSOA[district])
    print(f'In {district} there are {len(district_LSOA[district])} LSOAs')
print(f'In total there are {total} LSOAs in West Midlands Combined Authority')

In Birmingham there are 639 LSOAs
In Coventry there are 195 LSOAs
In Dudley there are 201 LSOAs
In Sandwell there are 186 LSOAs
In Solihull there are 134 LSOAs
In Walsall there are 167 LSOAs
In Wolverhampton there are 158 LSOAs
In total there are 1680 LSOAs in West Midlands Combined Authority


In [187]:
with open("output/districtLSOAs.json", "w") as outfile: 
    json.dump(district_LSOA, outfile, indent=2)

In [188]:
import json

json_data = json.dumps(district_LSOA, indent=2)  # Pretty print with indentation
print(json_data)

{
  "Birmingham": [
    "E01008881",
    "E01008882",
    "E01008883",
    "E01008884",
    "E01008885",
    "E01008886",
    "E01008887",
    "E01008888",
    "E01008889",
    "E01008890",
    "E01008891",
    "E01008892",
    "E01008893",
    "E01008894",
    "E01008895",
    "E01008896",
    "E01008897",
    "E01008898",
    "E01008899",
    "E01008901",
    "E01008905",
    "E01008906",
    "E01008907",
    "E01008909",
    "E01008910",
    "E01008911",
    "E01008913",
    "E01008914",
    "E01008915",
    "E01008916",
    "E01008917",
    "E01008918",
    "E01008919",
    "E01008920",
    "E01008921",
    "E01008923",
    "E01008924",
    "E01008925",
    "E01008927",
    "E01008928",
    "E01008929",
    "E01008930",
    "E01008931",
    "E01008932",
    "E01008933",
    "E01008934",
    "E01008935",
    "E01008936",
    "E01008937",
    "E01008938",
    "E01008939",
    "E01008940",
    "E01008941",
    "E01008942",
    "E01008943",
    "E01008944",
    "E01008945",
    "E01008

In [189]:
district_LSOA.keys()

dict_keys(['Birmingham', 'Coventry', 'Dudley', 'Sandwell', 'Solihull', 'Walsall', 'Wolverhampton'])

In [190]:
# engine = create_engine('sqlite:///crime_IMD.sqlite')
# final_merged.to_sql('crime_IMD', engine, index=False, if_exists='replace')

### initial approach was to use a geojson for all the merged data

this is done below but commented out as did not use

In [191]:
# engine = create_engine('sqlite:///crime_IMD.sqlite')

# print(engine)

In [192]:
# df = pd.read_sql("Select * from crime_IMD", con=engine, index_col=None)


In [193]:
# print(engine)

In [194]:

# Base = automap_base()
# Base.prepare(autoload_with=engine)


In [195]:
# session = Session(bind=engine)
# execute_string = "select * from crime_IMD"
# results = engine.connect().execute(text(execute_string)).fetchall()

In [196]:
# Base.classes.keys()

In [197]:
# inspector = inspect(engine)
# inspector.get_table_names()


In [198]:

# columns = inspector.get_columns('crime_IMD')
# for column in columns:
#     print(column["name"], column["type"])

In [199]:
# session.close()

In [200]:
# geojson = {
#     "type": "FeatureCollection",
#     "features": [
#         {
#             "type": "Feature",
#             "geometry": {
#                 "type": "Point",
#                 "coordinates": [str(Longitude), str(Latitude)],
#             },
#             "properties": {
#                 "Crime ID": str(crime_id),
#                 "LSOA code": str(LSOA_code),
#                 "LSOA name": str(LSOA_name),
#                 "LA District code": str(LA_District_code),
#                 "LA District name": str(LA_District_name),
#                 "Month": str(Month),
#                 "Crime type": str(crime_type),
#                 "IMD Score": str(IMD_Score),
#                 "IMD Decile": str(IMD_Decile),
#                 "Income Score": str(Income_Score),
#                 "Total population": str(Total_population)
#             },
#         } for crime_id, LSOA_code, LSOA_name, LA_District_code, LA_District_name, Month, crime_type, IMD_Score, IMD_Decile,  Income_Score, Total_population, Longitude, Latitude in results]
# }

In [201]:
# output_file_path = "crime_imd_merge.geojson"

# # Export GeoJSON to a file
# with open(output_file_path, "w") as output_file:
#     json.dump(geojson, output_file, indent=2)


### Aggregate required data and export as SQLite

In [202]:
final_merged2.head(2)

,crime_ID,LSOA_code,LSOA_name,LA_District_code,LA_District_name,Month,crime_type,Longitude,Latitude,IMD_Score,IMD_Decile,Income_Score,Total_population,lat_c,lon_c
0,id_1,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Anti-social behaviour,-1.841944,52.597265,19.086,5.0,0.124,1519.0,52.600857,-1.836545
1,id_2,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Burglary,-1.839063,52.597809,19.086,5.0,0.124,1519.0,52.600857,-1.836545


### Groupby Month and LSOA code

In [203]:
f = {'LSOA_code':'first','LSOA_name':'first','LA_District_name':'first','crime_ID':'count','Month':'first','IMD_Score':'first','IMD_Decile':'first','Income_Score':'first','Total_population':'first','lat_c':'first','lon_c':'first'}

month_lsoa_gb = final_merged2.groupby(['Month','LSOA_code']).agg(f)

print(type(month_lsoa_gb))
month_lsoa_gb.rename(columns={'crime_ID':'crime_count'}, inplace=True)
month_lsoa_gb


<class 'pandas.core.frame.DataFrame'>


LSOA_code        LSOA_name LA_District_name  crime_count  \
Month   LSOA_code                                                             
2020-10 E01008881  E01008881  Birmingham 067A       Birmingham           34   
        E01008882  E01008882  Birmingham 066A       Birmingham           22   
        E01008883  E01008883  Birmingham 078A       Birmingham           33   
        E01008884  E01008884  Birmingham 078B       Birmingham           23   
        E01008885  E01008885  Birmingham 076A       Birmingham           12   
...                      ...              ...              ...          ...   
2023-09 E01033643  E01033643  Birmingham 082F       Birmingham           21   
        E01033646  E01033646  Birmingham 031I       Birmingham           49   
        E01033648  E01033648  Birmingham 084F       Birmingham           21   
        E01033649  E01033649  Birmingham 058F       Birmingham            6   
        E01033650  E01033650  Birmingham 077F       Birmingham           17   

                     Month  IMD_Score  IMD_Decile  Income_Score  \
Month   LSOA_code                                                 
2020-10 E01008881  2020-10     41.179         2.0         0.232   
        E01008882  2020-10     59.693         1.0         0.311   
        E01008883  2020-10     38.636         2.0         0.230   
        E01008884  2020-10     44.315         1.0         0.273   
        E01008885  2020-10     22.921         4.0         0.102   
...                    ...        ...         ...           ...   
2023-09 E01033643  2023-09     50.599         1.0         0.333   
        E01033646  2023-09     64.138         1.0         0.292   
        E01033648  2023-09     55.778         1.0         0.351   
        E01033649  2023-09     52.028         1.0         0.322   
        E01033650  2023-09     52.488         1.0         0.359   

                   Total_population      lat_c     lon_c  
Month   LSOA_code                                         
2020-10 E01008881            1642.0  52.467331 -1.814266  
        E01008882            1769.0  52.466360 -1.828883  
        E01008883            2050.0  52.463228 -1.822689  
        E01008884            2091.0  52.461092 -1.835955  
        E01008885            1373.0  52.460640 -1.807253  
...                             ...        ...       ...  
2023-09 E01033643            3186.0  52.453954 -1.862875  
        E01033646            1891.0  52.518458 -1.850591  
        E01033648            2708.0  52.451622 -1.880321  
        E01033649            1881.0  52.482072 -1.846187  
        E01033650            2614.0  52.462104 -1.864970  

[60003 rows x 11 columns]

In [204]:
conn = sqlite3.connect('output/month_lsoa_gb.sqlite')
month_lsoa_gb.apply(lambda x: x.reset_index(drop=True)).to_sql(name='month_lsoa_gb', con=conn, index=False, if_exists='replace')

60003

In [205]:
query = 'SELECT * FROM month_lsoa_gb'
df = pd.read_sql(query, conn)
print(df)

       LSOA_code        LSOA_name LA_District_name  crime_count    Month  \
0      E01008881  Birmingham 067A       Birmingham           34  2020-10   
1      E01008882  Birmingham 066A       Birmingham           22  2020-10   
2      E01008883  Birmingham 078A       Birmingham           33  2020-10   
3      E01008884  Birmingham 078B       Birmingham           23  2020-10   
4      E01008885  Birmingham 076A       Birmingham           12  2020-10   
...          ...              ...              ...          ...      ...   
59998  E01033643  Birmingham 082F       Birmingham           21  2023-09   
59999  E01033646  Birmingham 031I       Birmingham           49  2023-09   
60000  E01033648  Birmingham 084F       Birmingham           21  2023-09   
60001  E01033649  Birmingham 058F       Birmingham            6  2023-09   
60002  E01033650  Birmingham 077F       Birmingham           17  2023-09   

       IMD_Score  IMD_Decile  Income_Score  Total_population      lat_c  \
0         41

In [206]:
df.head()

,LSOA_code,LSOA_name,LA_District_name,crime_count,Month,IMD_Score,IMD_Decile,Income_Score,Total_population,lat_c,lon_c
0,E01008881,Birmingham 067A,Birmingham,34,2020-10,41.179,2.0,0.232,1642.0,52.467331,-1.814266
1,E01008882,Birmingham 066A,Birmingham,22,2020-10,59.693,1.0,0.311,1769.0,52.466360,-1.828883
2,E01008883,Birmingham 078A,Birmingham,33,2020-10,38.636,2.0,0.230,2050.0,52.463228,-1.822689
3,E01008884,Birmingham 078B,Birmingham,23,2020-10,44.315,1.0,0.273,2091.0,52.461092,-1.835955
4,E01008885,Birmingham 076A,Birmingham,12,2020-10,22.921,4.0,0.102,1373.0,52.460640,-1.807253


In [207]:
df.tail()

,LSOA_code,LSOA_name,LA_District_name,crime_count,Month,IMD_Score,IMD_Decile,Income_Score,Total_population,lat_c,lon_c
59998,E01033643,Birmingham 082F,Birmingham,21,2023-09,50.599,1.0,0.333,3186.0,52.453954,-1.862875
59999,E01033646,Birmingham 031I,Birmingham,49,2023-09,64.138,1.0,0.292,1891.0,52.518458,-1.850591
60000,E01033648,Birmingham 084F,Birmingham,21,2023-09,55.778,1.0,0.351,2708.0,52.451622,-1.880321
60001,E01033649,Birmingham 058F,Birmingham,6,2023-09,52.028,1.0,0.322,1881.0,52.482072,-1.846187
60002,E01033650,Birmingham 077F,Birmingham,17,2023-09,52.488,1.0,0.359,2614.0,52.462104,-1.864970


In [208]:
engine = create_engine("sqlite:///output/month_lsoa_gb.sqlite")

Base = automap_base()
Base.prepare(autoload_with=engine)

session = Session(bind=engine)
execute_string = "select * from month_lsoa_gb"
results = engine.connect().execute(text(execute_string)).fetchall()

In [209]:
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns('month_lsoa_gb')
for column in columns:
    print(column["name"], column["type"])

LSOA_code TEXT
LSOA_name TEXT
LA_District_name TEXT
crime_count INTEGER
Month TEXT
IMD_Score REAL
IMD_Decile REAL
Income_Score REAL
Total_population REAL
lat_c REAL
lon_c REAL


In [210]:
session.close()

### Groupby crime type and Month

In [211]:
f = {'Month':'first','crime_type':'first','crime_ID':'count'}

crime_type_gb = final_merged2.groupby(['crime_type','Month']).agg(f)
crime_type_gb.rename(columns={'crime_ID':'crime_count'}, inplace=True)
crime_type_gb

Month                    crime_type  \
crime_type                   Month                                            
Anti-social behaviour        2020-10  2020-10         Anti-social behaviour   
                             2020-11  2020-11         Anti-social behaviour   
                             2020-12  2020-12         Anti-social behaviour   
                             2021-01  2021-01         Anti-social behaviour   
                             2021-02  2021-02         Anti-social behaviour   
...                                       ...                           ...   
Violence and sexual offences 2023-05  2023-05  Violence and sexual offences   
                             2023-06  2023-06  Violence and sexual offences   
                             2023-07  2023-07  Violence and sexual offences   
                             2023-08  2023-08  Violence and sexual offences   
                             2023-09  2023-09  Violence and sexual offences   

                                      crime_count  
crime_type                   Month                 
Anti-social behaviour        2020-10         5048  
                             2020-11         5284  
                             2020-12         3976  
                             2021-01         4778  
                             2021-02         3604  
...                                           ...  
Violence and sexual offences 2023-05        12420  
                             2023-06        12025  
                             2023-07        11890  
                             2023-08        10680  
                             2023-09        10913  

[504 rows x 3 columns]

In [212]:
crime_type_gb.index.unique


<bound method MultiIndex.unique of MultiIndex([(       'Anti-social behaviour', '2020-10'),
            (       'Anti-social behaviour', '2020-11'),
            (       'Anti-social behaviour', '2020-12'),
            (       'Anti-social behaviour', '2021-01'),
            (       'Anti-social behaviour', '2021-02'),
            (       'Anti-social behaviour', '2021-03'),
            (       'Anti-social behaviour', '2021-04'),
            (       'Anti-social behaviour', '2021-05'),
            (       'Anti-social behaviour', '2021-06'),
            (       'Anti-social behaviour', '2021-07'),
            ...
            ('Violence and sexual offences', '2022-12'),
            ('Violence and sexual offences', '2023-01'),
            ('Violence and sexual offences', '2023-02'),
            ('Violence and sexual offences', '2023-03'),
            ('Violence and sexual offences', '2023-04'),
            ('Violence and sexual offences', '2023-05'),
            ('Violence and sexual off

In [213]:
conn = sqlite3.connect('output/crime_type_count.sqlite')
crime_type_gb.apply(lambda x: x.reset_index(drop=True)).to_sql(name='crime_type_count', con=conn, index=False, if_exists='replace')

504

In [214]:
query = 'SELECT * FROM crime_type_count'
df = pd.read_sql(query, conn)
print(df)

       Month                    crime_type  crime_count
0    2020-10         Anti-social behaviour         5048
1    2020-11         Anti-social behaviour         5284
2    2020-12         Anti-social behaviour         3976
3    2021-01         Anti-social behaviour         4778
4    2021-02         Anti-social behaviour         3604
..       ...                           ...          ...
499  2023-05  Violence and sexual offences        12420
500  2023-06  Violence and sexual offences        12025
501  2023-07  Violence and sexual offences        11890
502  2023-08  Violence and sexual offences        10680
503  2023-09  Violence and sexual offences        10913

[504 rows x 3 columns]


In [215]:
df.head()

,Month,crime_type,crime_count
0,2020-10,Anti-social behaviour,5048
1,2020-11,Anti-social behaviour,5284
2,2020-12,Anti-social behaviour,3976
3,2021-01,Anti-social behaviour,4778
4,2021-02,Anti-social behaviour,3604


In [216]:

engine = create_engine("sqlite:///output/crime_type_count.sqlite")

Base = automap_base()
Base.prepare(autoload_with=engine)

session = Session(bind=engine)
execute_string = "select * from crime_type_count"
results = engine.connect().execute(text(execute_string)).fetchall()

In [217]:
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns('crime_type_count')
for column in columns:
    print(column["name"], column["type"])

Month TEXT
crime_type TEXT
crime_count INTEGER


In [218]:
session.close()

### Get coordinates for Heat Markers

In [219]:
final_merged2.head(2)

,crime_ID,LSOA_code,LSOA_name,LA_District_code,LA_District_name,Month,crime_type,Longitude,Latitude,IMD_Score,IMD_Decile,Income_Score,Total_population,lat_c,lon_c
0,id_1,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Anti-social behaviour,-1.841944,52.597265,19.086,5.0,0.124,1519.0,52.600857,-1.836545
1,id_2,E01009418,Birmingham 001B,E08000025,Birmingham,2020-10,Burglary,-1.839063,52.597809,19.086,5.0,0.124,1519.0,52.600857,-1.836545


In [220]:
heat_markers = final_merged2[['Month','Latitude','Longitude']]
heat_markers.head()

,Month,Latitude,Longitude
0,2020-10,52.597265,-1.841944
1,2020-10,52.597809,-1.839063
2,2020-10,52.597265,-1.841944
3,2020-10,52.597265,-1.841944
4,2020-10,52.597809,-1.839063


In [221]:
len(heat_markers)

1124466

In [222]:
conn = sqlite3.connect('output/heat_markers.sqlite')
heat_markers.apply(lambda x: x.reset_index(drop=True)).to_sql(name='heat_markers', con=conn, index=False, if_exists='replace')

1124466

In [223]:
query = 'SELECT * FROM heat_markers'
df = pd.read_sql(query, conn)
print(df)

           Month   Latitude  Longitude
0        2020-10  52.597265  -1.841944
1        2020-10  52.597809  -1.839063
2        2020-10  52.597265  -1.841944
3        2020-10  52.597265  -1.841944
4        2020-10  52.597809  -1.839063
...          ...        ...        ...
1124461  2023-09  52.379433  -1.519938
1124462  2023-09  52.378606  -1.520035
1124463  2023-09  52.378125  -1.523360
1124464  2023-09  52.378125  -1.523360
1124465  2023-09  52.377078  -1.517936

[1124466 rows x 3 columns]


In [224]:

engine = create_engine("sqlite:///output/heat_markers.sqlite")

Base = automap_base()
Base.prepare(autoload_with=engine)

session = Session(bind=engine)
execute_string = "select * from heat_markers"
results = engine.connect().execute(text(execute_string)).fetchall()

In [225]:
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns('heat_markers')
for column in columns:
    print(column["name"], column["type"])

Month TEXT
Latitude REAL
Longitude REAL


In [226]:
session.close()